## Importing accidents data flow.

This notebook processing FARS data (accident level granularity) into single csv file for all years between 1975-2016.
ETL is smart enough =) to parse year level folders for accidents, vehicles, persons, convert metadata-columns to be the same for all years and enrich data values like weather type, road type, state name, etc. based on lookups which also different from year to year. 

### Prerequirements:

- downloading fars data, unzipping and placing into rootDir, like rootDir+'/1975', etc. (should be automated) 

In [1]:
import warnings
warnings.filterwarnings("ignore")

from string import ascii_letters
import numpy as np
import pandas as pd

from dbfread import DBF
from pandas import DataFrame

import sys
import datetime
import os
import os.path
import zipfile
import shutil

In [2]:
baseDir = '/media/viktor/1E65-F3A5'
rootDir = baseDir+'/fars'  # folder with data by year, should be pre-downloaded and uziped - around 6.5 Gb
unzipedFilesTemp=baseDir+'/fars-unziped'
targetDir=baseDir+'/fars-all/'

yearList=list(range(1975,2016))
#print(yearList)

# for processing files
dir=targetDir

In [3]:
# List of dataframes to load data in 

df_acc={}
df_veh={}
df_per={}

df_acc_norm={}
df_veh_norm={}
df_per_norm={}

In [4]:
for eachYear in yearList:

    zipFile=''
    raw_masks=['FARS'+str(eachYear)+".zip",'FARSDBF'+str(eachYear)[2:4]+".zip",'FARS'+str(eachYear)+"NationalDBF.zip", 'Auxiliary_FARS_DBF_'+str(eachYear)+'.ZIP']

    for dirpath, dirnames, filenames in os.walk(rootDir):
        #print(filenames)
        for filename in [f for f in filenames if (f in raw_masks)]:
            zipFile=os.path.join(dirpath, filename)

    print(zipFile)

    if zipFile!='':

        with zipfile.ZipFile(zipFile, "r") as zip_ref:
            zip_ref.extractall(unzipedFilesTemp)

        types=[["ACC_AUX.dbf", "acc"+str(eachYear)+".dbf", "ACCIDENT.dbf", "accident.dbf", "ACCIDENT.DBF", "accident.DBF"],
               ["VEH_AUX.dbf", "veh" + str(eachYear) + ".dbf", "VEHICLE.dbf", "vehicle.dbf", "VEHICLE.DBF", "vehicle.DBF"],
               ["PER_AUX.dbf", "per" + str(eachYear) + ".dbf", "PERSON.dbf", "person.dbf", "PERSON.DBF", "person.DBF"]
             ]

        for type in types:

            for dirpath, dirnames, filenames in os.walk(unzipedFilesTemp):
                for filename in [f for f in filenames if(f in type)]:
                    typeFile = os.path.join(dirpath, filename)
                    print(typeFile)
                    shutil.move(typeFile, targetDir+'/'+str(filename.lower()[0:3])+'_'+str(eachYear)+'.dbf')


/media/viktor/1E65-F3A5/fars/1975/DBF/FARS1975.zip
/media/viktor/1E65-F3A5/fars-unziped/ACCIDENT.dbf
/media/viktor/1E65-F3A5/fars-unziped/VEHICLE.dbf
/media/viktor/1E65-F3A5/fars-unziped/PERSON.dbf
/media/viktor/1E65-F3A5/fars/1976/DBF/FARS1976.zip
/media/viktor/1E65-F3A5/fars-unziped/ACCIDENT.dbf
/media/viktor/1E65-F3A5/fars-unziped/VEHICLE.dbf
/media/viktor/1E65-F3A5/fars-unziped/PERSON.dbf
/media/viktor/1E65-F3A5/fars/1977/DBF/FARS1977.zip
/media/viktor/1E65-F3A5/fars-unziped/ACCIDENT.dbf
/media/viktor/1E65-F3A5/fars-unziped/VEHICLE.dbf
/media/viktor/1E65-F3A5/fars-unziped/PERSON.dbf
/media/viktor/1E65-F3A5/fars/1978/DBF/FARS1978.zip
/media/viktor/1E65-F3A5/fars-unziped/ACCIDENT.dbf
/media/viktor/1E65-F3A5/fars-unziped/VEHICLE.dbf
/media/viktor/1E65-F3A5/fars-unziped/PERSON.dbf
/media/viktor/1E65-F3A5/fars/1979/DBF/FARS1979.zip
/media/viktor/1E65-F3A5/fars-unziped/accident.dbf
/media/viktor/1E65-F3A5/fars-unziped/vehicle.dbf
/media/viktor/1E65-F3A5/fars-unziped/person.dbf
/media/vik

/media/viktor/1E65-F3A5/fars-unziped/ACC_AUX.dbf
/media/viktor/1E65-F3A5/fars-unziped/accident.dbf
/media/viktor/1E65-F3A5/fars-unziped/VEH_AUX.dbf
/media/viktor/1E65-F3A5/fars-unziped/vehicle.dbf
/media/viktor/1E65-F3A5/fars-unziped/PER_AUX.dbf
/media/viktor/1E65-F3A5/fars-unziped/person.dbf


In [5]:
for eachYear in yearList:
    
    dbf = DBF(dir+'acc_'+str(eachYear)+'.dbf',encoding='iso-8859-1')
    df_acc_temp = DataFrame(iter(dbf))
    df_acc[eachYear]=df_acc_temp
    #print(str(eachYear)+":"+str(df_acc_temp.shape))

In [6]:
for eachYear in yearList:
    
    dbf = DBF(dir+'veh_'+str(eachYear)+'.dbf',encoding='iso-8859-1')
    df_veh_temp = DataFrame(iter(dbf))
    df_veh[eachYear]=df_veh_temp
    #print(df_veh_temp.shape)

In [7]:
for eachYear in yearList:
    
    dbf = DBF(dir+'per_'+str(eachYear)+'.dbf',encoding='iso-8859-1')
    df_per_temp = DataFrame(iter(dbf))
    df_per[eachYear]=df_per_temp
    #print(df_per_temp.shape)

### Processing data - accidents level

In [8]:
for eachYear in yearList:
    
    ## add custom columns mappings based on the year specific details
    
    # ------------ Year specific value fixes --------------
    
    if eachYear<=1997:
            df_acc[eachYear]['YEAR']=df_acc[eachYear]['YEAR'].apply(lambda x: x+1900 if x<100 else x)
            
    if eachYear<=1986 and 'CL_TWAY' in df_acc[eachYear]:
            df_acc[eachYear]['RoadType']=df_acc[eachYear]['CL_TWAY']
    else:
        if eachYear>=1987 and 'ROUTE' in df_acc[eachYear]:
            df_acc[eachYear]['RoadType']=df_acc[eachYear]['ROUTE']
        else:
            df_acc[eachYear]['RoadType']=np.nan
    
    # ----------- Custom mappings ---------------
    
    if 'VE_TOTAL' in df_acc[eachYear]:
        
        df_acc[eachYear]['Vehicle_count']=df_acc[eachYear]['VE_TOTAL']
        
        if ('VEHICLES' in df_acc[eachYear]) :
            if df_acc[eachYear]['VEHICLES'] is not None:
                df_acc[eachYear]['MotorVehicle_count']=df_acc[eachYear]['VEHICLES']
            else:
                if 'VE_FORMS' in df_acc[eachYear]:
                    df_acc[eachYear]['MotorVehicle_count']=df_acc[eachYear]['VE_FORMS']
        else:
            if 'VE_FORMS' in df_acc[eachYear]:
                df_acc[eachYear]['MotorVehicle_count']=df_acc[eachYear]['VE_FORMS']
            
    else:
        if 'VEHICLES' in df_acc[eachYear] and df_acc[eachYear]['VEHICLES'] is not None:
            df_acc[eachYear]['MotorVehicle_count']=df_acc[eachYear]['VEHICLES']
            df_acc[eachYear]['Vehicle_count']=df_acc[eachYear]['VEHICLES']
        else:
            if 'VE_FORMS' in df_acc[eachYear]:
                df_acc[eachYear]['MotorVehicle_count']=df_acc[eachYear]['VE_FORMS']
                df_acc[eachYear]['Vehicle_count']=df_acc[eachYear]['VE_FORMS']
    
    # ------------ General mappings --------------
    
    df_acc[eachYear].rename(columns={'COUNTY': 'COUNTRY'
                            , 'PVH_INVL': 'ParkedVehicle_count'
                            , 'PERNOTMVIT': 'PersonsOutsideVehicle_count'
                            , 'PERSONS': 'PersonsInVehicles_count'
                            , 'PEDS': 'Pedestrians_count'
                            , 'DRUNK_DR': 'DrunkDriver_count'
                            , 'FATALS': 'Fatals_count'
                            , 'LGT_COND':'LightCondition'
                            , 'REL_ROAD':'RelationToTrafficway'
                            , 'MAN_COLL':'MannerOfCollision'
                            , 'SCH_BUS':'SchoolBusRelated'
                            , 'TYP_INT':'TypeOfIntersaction'
                            , 'WEATHER':'Weather'
                            , 'NHS':'NationalHighwaySystem'
                            , 'FUNC_SYS':'FunctionalSystem'
                            , 'CF1':'CrashConditions1'
                            , 'CF2':'CrashConditions2'
                            , 'CF3':'CrashConditions3'
                            , 'HOUR':'Hour'
                            , 'MINUTE':'Minute'
                            , 'DAY_WEEK':'DayOfWeek'
                            , 'LATITUDE':'Latitude'
                            , 'LONGITUD':'Longitude'
                            , 'HOSP_HR':'HourArrivingHospital'
                            , 'HOSP_MN':'MinuteArrivingHospital'}, inplace=True)
    

In [9]:
## adding missing columns
acc_col_list=['ST_CASE','STATE', 'COUNTRY', 'CITY', 'YEAR', 'MONTH','DAY'
                            , 'Hour','Minute','DayOfWeek'
                            ,'Latitude','Longitude'
                            ,'LightCondition'
                            ,'RelationToTrafficway'
                            ,'MannerOfCollision'
                            ,'SchoolBusRelated'
                            ,'TypeOfIntersaction'
                            ,'Weather'
                            ,'NationalHighwaySystem'
                            ,'FunctionalSystem'
                            ,'RoadType'
                            ,'CrashConditions1'
                            ,'CrashConditions2'
                            ,'CrashConditions3'
                            ,'Vehicle_count'
                            ,'MotorVehicle_count'
                            ,'ParkedVehicle_count'
                            ,'PersonsOutsideVehicle_count'
                            ,'PersonsInVehicles_count'
                            ,'Pedestrians_count'
                            ,'DrunkDriver_count'
                            ,'Fatals_count']
for eachYear in yearList:
    for col in acc_col_list:
        if col in df_acc[eachYear]:
            continue
        else:
            df_acc[eachYear][col]=np.nan

In [10]:
## making key column & convert types

for eachYear in yearList:

    for col in acc_col_list:
        df_acc[eachYear][col] = df_acc[eachYear][col].apply(lambda x: int(x) if (x == x) and (x is not None) else np.nan)
    
    df_acc[eachYear]['CASE_NUMBER'] = df_acc[eachYear]["ST_CASE"].map(str)+"-"+ df_acc[eachYear]["STATE"].map(str)+"-"+  df_acc[eachYear]["COUNTRY"].map(str)+"-"+  df_acc[eachYear]["CITY"].map(str)+"-"+  df_acc[eachYear]["YEAR"].map(str)+"-"+  df_acc[eachYear]["MONTH"].map(str)+"-"+  df_acc[eachYear]["DAY"].map(str)

In [11]:
## creating order list

for eachYear in yearList:
    df_acc_norm[eachYear]=df_acc[eachYear][acc_col_list]  


In [12]:
## TODO: prepare aggregations and join to frame: like number of accidents per year/per month/etc.

In [13]:
## Map categorigal values ids to values

'''
                            #State, Country, City
                            #,'LightCondition'
                            --,'RelationToTrafficway'
                            #,'MannerOfCollision'
                            #,'SchoolBusRelated'
                            #,'TypeOfIntersaction'
                            #,'Weather'
                            #,'NationalHighwaySystem'
                            --,'FunctionalSystem'
                            --,'RouteSigning'
                            --,'RoadOwnership'
                            #,'RoadType'
                            #,'CrashConditions1'
                            #,'CrashConditions2'
                            #,'CrashConditions3'
'''

stateNames = {1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas',6: 'California', 8: 'Colorado', 9: 'Connecticut', 10: 'Delaware',           11: 'District of Columbia', 12: 'Florida', 13: 'Georgia', 15: 'Hawaii',16: 'Idaho', 17: 'Illinois', 18: 'Indiana', 19: 'Iowa', 20: 'Kansas',21: 'Kentucky', 22: 'Louisiana', 23: 'Maine', 24: 'Maryland',           25: 'Massachusetts', 26: 'Michigan', 27: 'Minnesota',           28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska',           32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico',           36: 'New York', 37: 'North Carolina', 38: 'North Dakota', 39: 'Ohio',           40: 'Oklahoma', 41: 'Oregon', 42: 'Pennsylvania', 43: 'Puerto Rico',           44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota', 47: 'Tennessee',           48: 'Texas', 49: 'Utah', 50: 'Vermont', 51: 'Virginia', 52: 'Virgin Islands',           53: 'Washington', 54: 'West Virginia', 55: 'Wisconsin', 56: 'Wyoming'}

roadTypes80={0:'N/A',1:'Interstate',2:'Other Limited Access',3:'Other US Route',4:'Other State Route',5:'Other Major Artery',6:'Country Road',7:'Local Street',8:'Other Road',9:'Unknown'}
roadTypes87={0:'N/A',1:'Interstate',2:'Other US Route',3:'Other State Route',4:'Country Road',5:'Local Street',6:'Other Road',7:'Unknown',8:'Unknown',9:'Unknown'}
roadTypes16={0:'N/A',1:'Interstate',2:'US Highway',3:'State Highway',4:'Country Road',5:'Local Street-Township',6:'Local Street-Municipality',7:'Local Street-Frontage Road',8:'Other Road',9:'Unknown'}

nationalHighwaySystem={0:'Not National Highway System',1:'National Highway System',9:'Unknown'}

crashConditionBefore81={0:'None',1:'Rain/Snow/Fog/Smoke/Sand',2:'Reflected Glare / Bright Sunlight / HeadLights',3:'Curve / Hill or other design',4:'Building/Billboard',5:'Trees/Crops/Vegetation',6:'Moving vehicle',7:'Parked Vehicle',8:'Other not classified object',20:'Severe crosswind',21:'Wind form passing truck',22:'Slippery Surface',23:'Avoiding Debris',24:'Ruts/Holes/Bumps in Road',25:'Avoiding animals',26:'Avoiding vehicle',27:'Avoiding phantom veh.',28:'Avoiding pedestrian,bycicle',29:'Avoiding water/snow',40:'Traffic Controlls not functioning properly',41:'Inadequate warning of exits',42:'Uncontroller Intersection',43:'Shoulder too low or high',44:'Shoulders too narrow',47:'Other construction',48:'No or obscured Pavement Markers',49:'Surface Underwater',50:'Inadequate construction or poor design',51:'Surface washed out',99:'N/A',46:'N/A',45:'N/A'}
crashCondition82_12={0:'None',1:'Inadequate warning of exits',2:'Shoulder related',3:'Other maintainance or construction',4:'No or obscured Pavement Markers',5:'Surface Underwater',6:'Inadequate construction or poor design',7:'Surface washed out',13:'Aggressive driving',14:'Motor vehicle falling cargo',15:'Non-occupant struck by falling cargo',16:'Non-occupant struck vehicle',17:'Vehicle set in motion by non-driver',18:'Date of crash not the same date',19:'Recent previous scene crash nearby',20:'Police pursuit involved',21:'Within designated school zone',22:'Speed limit',23:'Identification of stalled vehicle',24:'Unstabillized situation began',25:'Toll plaza related',99:'N/A',46:'N/A',45:'N/A'}
crashConditionFrom13={0:'None',1:'Inadequate warning of exits',2:'Shoulder related',3:'Other maintainance or construction',4:'No or obscured Pavement Markers',5:'Surface Underwater',6:'Inadequate construction or poor design',7:'Surface washed out',13:'Aggressive driving',14:'Motor vehicle falling cargo',15:'Non-occupant struck by falling cargo',16:'Non-occupant struck vehicle',17:'Vehicle set in motion by non-driver',18:'Date of crash not the same date',19:'Recent previous scene crash nearby',20:'Police pursuit involved',21:'Within designated school zone',22:'Speed limit',23:'Identification of stalled vehicle',24:'Unstabillized situation began',25:'Toll plaza related',26:'Backup due to prior not recurring incident',27:'Backup due to prior crash',28:'Backup due to regular congestion',99:'N/A',46:'N/A',45:'N/A'}

schoolBus={0:'No',1:'Yes',8:'Not Reported'}

weatherMap75_79={1:'Clear',2:'Rain',3:'Sleet',4:'Snow',5:'N/A',6:'N/A',7:'Cloudy',8:'N/A',9:'Unknown'}
weatherMap80_81={1:'Normal',2:'Rain',3:'Sleet',4:'Snow',5:'Fog',6:'N/A',7:'N/A',8:'Smog/Smoke',9:'Unknown'}
weatherMap82_06={1:'No adverse conditions',2:'Rain',3:'Sleet',4:'Snow',5:'Fog',6:'Rain and Fog',7:'Sleet and Fog',8:'Smog/Smoke',9:'Unknown'}
weatherMap07_09={0:'No adverse conditions',1:'Clear',2:'Rain',3:'Sleet',4:'Snow',5:'Fog/Smog/Smoke',6:'Severe crosswinds',7:'Blowing sand, soil, dirt',8:'Other',9:'Unknown'}
weatherMap10_12={0:'No addictional conditions',1:'Clear',2:'Rain',3:'Sleet',4:'Snow',5:'Fog/Smog/Smoke',6:'Severe crosswinds',7:'Blowing sand, soil, dirt',8:'Other',9:'Unknown',10:'Cloudly',11:'Blowing Snow',98:'Not reported',99:'Unknown'}
weatherMap13_17={0:'No addictional conditions',1:'Clear',2:'Rain',3:'Sleet',4:'Snow',5:'Fog/Smog/Smoke',6:'Severe crosswinds',7:'Blowing sand, soil, dirt',8:'Other',9:'Unknown',10:'Cloudly',11:'Blowing Snow',12:'Freezing rain or drizzle',98:'Not reported',99:'Unknown'}

lightCondMap75_79={1:'DayLight',2:'Dark',3:'Dark but lighted',6:'Dawn or dusk',9:'Unknown'}
lightCondMap80_08={1:'DayLight',2:'Dark',3:'Dark but lighted',4:'Dawn',5:'Dusk',9:'Unknown'}
lightCondMap09={1:'DayLight',2:'Dark - not lighted',3:'Dark but lighted',4:'Dawn',5:'Dusk',6:'Dark - unknown lighting',9:'Unknown'}
lightCondMap10_17={1:'DayLight',2:'Dark - not lighted',3:'Dark but lighted',4:'Dawn',5:'Dusk',6:'Dark - unknown lighting',8:'Not reported',9:'Unknown'}

typeOfIntersactionMap={1:'Not an intersaction',2:'Four-Way intesaction',3:'T-intersaction',4:'Y-intersaction',5:'Traffic circle',6:'Roundabout',7:'Five-point or more',8:'Not reported',98:'Not reported',9:'Unknown',99:'Unknown',10:'L-intersaction'}

manColisMap75_01={0:'Not collision',1:'Rear-End',2:'Head-On',3:'Rear-to-rear',4:'Angle',5:'Sideswipe same direction',6:'Sideswipe opposite direction',7:'Sideswipe',9:'Unknown'}
manColisMap02_09={0:'Not collision',1:'Rear-End',2:'Head-On',3:'Angle front to side same direction',4:'Angle front to side opposite direction',5:'Right Angle',6:'Angle',7:'Sideswipe same direction',8:'Sideswipe opposite direction',9:'Rear-to-side',10:'Rear-to-rear',11:'Other',98:'Not reported', 99:'Unknown'}
manColisMap10_17={0:'Not collision',1:'Rear-End',2:'Head-On',6:'Angle',7:'Sideswipe same direction',8:'Sideswipe opposite direction',9:'Rear-to-side',10:'Rear-to-rear',11:'Other',98:'Not reported', 99:'Unknown'}


#---------- apply ---------

for eachYear in yearList:
    df_acc_norm[eachYear]['stateName']=df_acc_norm[eachYear]['STATE'].apply(lambda x: stateNames[x])
    
    df_acc_norm[eachYear]['RoadType']=df_acc_norm[eachYear]['RoadType'].fillna(0)
    if eachYear>=1975 and eachYear<=1980:
        df_acc_norm[eachYear]['RoadTypeName']=df_acc_norm[eachYear]['RoadType'].apply(lambda x: roadTypes80[x])
    if eachYear>1980 and eachYear<=1986:
        df_acc_norm[eachYear]['RoadTypeName']=df_acc_norm[eachYear]['RoadType'].apply(lambda x: roadTypes87[x])
    if eachYear>1986 and eachYear<=2017:
        df_acc_norm[eachYear]['RoadTypeName']=df_acc_norm[eachYear]['RoadType'].apply(lambda x: roadTypes16[x])
            
    df_acc_norm[eachYear]['NationalHighwaySystem']=df_acc_norm[eachYear]['NationalHighwaySystem'].fillna(9)
    df_acc_norm[eachYear]['NationalHighwaySystemName']=df_acc_norm[eachYear]['NationalHighwaySystem'].apply(lambda x: nationalHighwaySystem[x])

    df_acc_norm[eachYear]['CrashConditions1']=df_acc_norm[eachYear]['CrashConditions1'].fillna(99)
    df_acc_norm[eachYear]['CrashConditions2']=df_acc_norm[eachYear]['CrashConditions2'].fillna(99)
    df_acc_norm[eachYear]['CrashConditions3']=df_acc_norm[eachYear]['CrashConditions3'].fillna(99)

    if eachYear>=1975 and eachYear<=1981:
        df_acc_norm[eachYear]['CrashConditionsName1']=df_acc_norm[eachYear]['CrashConditions1'].apply(lambda x: crashConditionBefore81[x] if x in crashConditionBefore81.keys() else 'N/A')
        df_acc_norm[eachYear]['CrashConditionsName2']=df_acc_norm[eachYear]['CrashConditions2'].apply(lambda x: crashConditionBefore81[x] if x in crashConditionBefore81.keys() else 'N/A')
        df_acc_norm[eachYear]['CrashConditionsName3']=df_acc_norm[eachYear]['CrashConditions3'].apply(lambda x: crashConditionBefore81[x] if x in crashConditionBefore81.keys() else 'N/A')
    if eachYear>1981 and eachYear<=2012:
        df_acc_norm[eachYear]['CrashConditionsName1']=df_acc_norm[eachYear]['CrashConditions1'].apply(lambda x: crashCondition82_12[x] if x in crashCondition82_12.keys() else 'N/A')
        df_acc_norm[eachYear]['CrashConditionsName2']=df_acc_norm[eachYear]['CrashConditions2'].apply(lambda x: crashCondition82_12[x] if x in crashCondition82_12.keys() else 'N/A')
        df_acc_norm[eachYear]['CrashConditionsName3']=df_acc_norm[eachYear]['CrashConditions3'].apply(lambda x: crashCondition82_12[x] if x in crashCondition82_12.keys() else 'N/A')
    if eachYear>=2013 and eachYear<=2017:
        df_acc_norm[eachYear]['CrashConditionsName1']=df_acc_norm[eachYear]['CrashConditions1'].apply(lambda x: crashConditionFrom13[x] if x in crashConditionFrom13.keys() else 'N/A')
        df_acc_norm[eachYear]['CrashConditionsName2']=df_acc_norm[eachYear]['CrashConditions2'].apply(lambda x: crashConditionFrom13[x] if x in crashConditionFrom13.keys() else 'N/A')
        df_acc_norm[eachYear]['CrashConditionsName3']=df_acc_norm[eachYear]['CrashConditions3'].apply(lambda x: crashConditionFrom13[x] if x in crashConditionFrom13.keys() else 'N/A')

    df_acc_norm[eachYear]['SchoolBusRelated']=df_acc_norm[eachYear]['SchoolBusRelated'].fillna(8)
    df_acc_norm[eachYear]['SchoolBusRelatedName']=df_acc_norm[eachYear]['SchoolBusRelated'].apply(lambda x: schoolBus[x])


    if eachYear>=1975 and eachYear<=1979:
        df_acc_norm[eachYear]['WeatherName']=df_acc_norm[eachYear]['Weather'].apply(lambda x: weatherMap75_79[x] if x in weatherMap75_79.keys() else 'N/A')
    if eachYear>=1980 and eachYear<=1981:
        df_acc_norm[eachYear]['WeatherName']=df_acc_norm[eachYear]['Weather'].apply(lambda x: weatherMap80_81[x] if x in weatherMap80_81.keys() else 'N/A')
    if eachYear>=1982 and eachYear<=2006:
        df_acc_norm[eachYear]['WeatherName']=df_acc_norm[eachYear]['Weather'].apply(lambda x: weatherMap82_06[x] if x in weatherMap82_06.keys() else 'N/A')
    if eachYear>=2007 and eachYear<=2009:
        df_acc_norm[eachYear]['WeatherName']=df_acc_norm[eachYear]['Weather'].apply(lambda x: weatherMap07_09[x] if x in weatherMap07_09.keys() else 'N/A')
    if eachYear>=2010 and eachYear<=2012:
        df_acc_norm[eachYear]['WeatherName']=df_acc_norm[eachYear]['Weather'].apply(lambda x: weatherMap10_12[x] if x in weatherMap10_12.keys() else 'N/A')
    if eachYear>=2013 and eachYear<=2017:
        df_acc_norm[eachYear]['WeatherName']=df_acc_norm[eachYear]['Weather'].apply(lambda x: weatherMap13_17[x] if x in weatherMap13_17.keys() else 'N/A')

    df_acc_norm[eachYear]['LightConditionName']=np.nan    
    if eachYear>=1975 and eachYear<=1979:
        df_acc_norm[eachYear]['LightConditionName']=df_acc_norm[eachYear]['LightCondition'].apply(lambda x: lightCondMap75_79[x] if x in lightCondMap75_79.keys() else 'N/A')
    if eachYear>=1980 and eachYear<=2008:
        df_acc_norm[eachYear]['LightConditionName']=df_acc_norm[eachYear]['LightCondition'].apply(lambda x: lightCondMap80_08[x] if x in lightCondMap80_08.keys() else 'N/A')
    if eachYear==2009:
        df_acc_norm[eachYear]['LightConditionName']=df_acc_norm[eachYear]['LightCondition'].apply(lambda x: lightCondMap09[x] if x in lightCondMap09.keys() else 'N/A')
    if eachYear>=2013 and eachYear<=2017:
        df_acc_norm[eachYear]['LightConditionName']=df_acc_norm[eachYear]['LightCondition'].apply(lambda x: lightCondMap10_17[x] if x in lightCondMap10_17.keys() else 'N/A')

    df_acc_norm[eachYear]['TypeOfIntersactionName']=np.nan
    df_acc_norm[eachYear]['TypeOfIntersaction']=df_acc_norm[eachYear]['TypeOfIntersaction'].fillna(99)
    df_acc_norm[eachYear]['TypeOfIntersactionName']=df_acc_norm[eachYear]['TypeOfIntersaction'].apply(lambda x: typeOfIntersactionMap[x] if x in typeOfIntersactionMap.keys() else 'N/A')

    df_acc_norm[eachYear]['MannerOfCollisionName']=np.nan    
    if eachYear>=1975 and eachYear<=2001:
        df_acc_norm[eachYear]['MannerOfCollisionName']=df_acc_norm[eachYear]['MannerOfCollision'].apply(lambda x: manColisMap75_01[x] if x in manColisMap75_01.keys() else 'N/A')
    if eachYear>=2002 and eachYear<=2009:
        df_acc_norm[eachYear]['MannerOfCollisionName']=df_acc_norm[eachYear]['MannerOfCollision'].apply(lambda x: manColisMap02_09[x] if x in manColisMap02_09.keys() else 'N/A')
    if eachYear>=2010 and eachYear<=2017:
        df_acc_norm[eachYear]['MannerOfCollisionName']=df_acc_norm[eachYear]['MannerOfCollision'].apply(lambda x: manColisMap10_17[x] if x in manColisMap10_17.keys() else 'N/A')

        

In [14]:
## Append all years together
df_acc_all=df_acc_norm[1975]

for col in acc_col_list:
        df_acc_all[col] = df_acc_all[col].apply(lambda x: int(int(x)) if (x == x) and (x is not None) else np.nan)

for eachYear in yearList:
    if eachYear!=2015 :
        #print(eachYear+1)
        df_acc_all=df_acc_all.append(df_acc_norm[eachYear+1])
    

In [15]:
df_acc_all.shape

(1546170, 43)

In [16]:
df_acc_all.head(100)

,ST_CASE,STATE,COUNTRY,CITY,YEAR,MONTH,DAY,Hour,Minute,DayOfWeek,...,RoadTypeName,NationalHighwaySystemName,CrashConditionsName1,CrashConditionsName2,CrashConditionsName3,SchoolBusRelatedName,WeatherName,LightConditionName,TypeOfIntersactionName,MannerOfCollisionName
0,10001,1,95,400.0,1975,1.0,4.0,22.0,24.0,7.0,...,Other US Route,Unknown,None,None,None,Not Reported,Clear,Dark,Unknown,Head-On
1,10002,1,95,110.0,1975,1.0,2.0,4.0,30.0,5.0,...,Other State Route,Unknown,None,None,None,Not Reported,Clear,Dark,Unknown,Not collision
2,10003,1,35,0.0,1975,1.0,1.0,4.0,0.0,4.0,...,Country Road,Unknown,None,None,None,Not Reported,Clear,Dark,Unknown,Not collision
3,10004,1,77,0.0,1975,1.0,2.0,14.0,50.0,5.0,...,Country Road,Unknown,None,None,None,Not Reported,Clear,DayLight,Unknown,Not collision
4,10005,1,97,2100.0,1975,1.0,5.0,20.0,0.0,1.0,...,Local Street,Unknown,None,None,None,Not Reported,Clear,Dark,Unknown,Not collision
5,10006,1,65,0.0,1975,1.0,4.0,15.0,0.0,7.0,...,Other State Route,Unknown,None,None,None,Not Reported,Clear,DayLight,Unknown,Angle
6,10007,1,89,2225.0,1975,1.0,8.0,13.0,20.0,4.0,...,Local Street,Unknown,None,None,None,Not Reported,Rain,DayLight,Unknown,Not collision
7,10008,1,69,2961.0,1975,1.0,4.0,1.0,15.0,7.0,...,Local Street,Unknown,None,None,None,Not Reported,Rain,Dark,Unknown,Not collision
8,10009,1,73,1270.0,1975,1.0,6.0,17.0,40.0,2.0,...,Other US Route,Unknown,None,None,None,Not Reported,Clear,Dawn or dusk,Unknown,Not collision
9,10010,1,101,2130.0,1975,1.0,3.0,17.0,44.0,6.0,...,Other US Route,Unknown,Rain/Snow/Fog/Smoke/Sand,None,None,Not Reported,Rain,Dark,Unknown,Angle


In [17]:
# Just to look at some values, for validation

with pd.option_context('display.max_rows', None, 'display.max_columns', 195):
    print(df_acc_all[df_acc_all.ST_CASE==10012])

    ST_CASE  STATE  COUNTRY    CITY  YEAR  MONTH   DAY  Hour  Minute  \
11    10012      1       47     0.0  1975    1.0   5.0  17.0    48.0   
11    10012      1       65     0.0  1976    1.0  17.0  22.0    10.0   
11    10012      1      109     0.0  1977    1.0  14.0   5.0     0.0   
11    10012      1       97  2100.0  1978    1.0  11.0  12.0    10.0   
10    10012      1        9  2330.0  1979    1.0   6.0  13.0    10.0   
11    10012      1       77     0.0  1980    1.0   5.0   0.0    50.0   
11    10012      1       23     0.0  1981    1.0   3.0  21.0    20.0   
11    10012      1      121  2900.0  1982    1.0   8.0  16.0     8.0   
11    10012      1       73   350.0  1983    1.0  10.0  14.0     0.0   
11    10012      1       55     0.0  1984    1.0  14.0   2.0    10.0   
10    10012      1       97     0.0  1985    1.0  11.0  23.0    45.0   
10    10012      1       17     0.0  1986    1.0  12.0   4.0     0.0   
11    10012      1       43     0.0  1987    1.0  15.0   6.0    

In [18]:
df_acc_all.to_csv(baseDir+"/Accidents1975_2015.csv")

In [19]:
## Finished with accidents